# 00 정형_횡단면

# 1. 분석 환경 준비¶
## 1.1. 데이터 불러오기

분석하려는 데이터를 가져오는 작업

- 파이썬 라이브러리 Pandas 이용

- read_csv 이용

In [ ]:
import pandas as pd 

df = pd.read_csv('data/train.csv')

# 2. 데이터 전처리

## 2.1. Imputation

In [ ]:
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(df)

### 2.1.1. 결측시 있을시

* 결측치가 있는 row 확인

In [ ]:
df[df.isna().sum(axis=1) > 0]

* 결측치 처리

카테고리형 feature가 결측치인 경우 : 해당 행들을 삭제   
수치형 feature가 결측치인 경우 : 평균값을 채워줌  

In [ ]:
# 결측치를 처리하는 함수를 작성합니다.
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        if dtype == 'O':
            # 카테고리형 feature가 결측치인 경우 해당 행들을 삭제해 주었습니다.
            temp = temp.dropna(subset=[col])
        elif dtype == int or dtype == float:
            # 수치형 feature가 결측치인 경우 평균값을 채워주었습니다.
            temp.loc[:,col] = temp[col].fillna(temp[col].mean())
    return temp

df = handle_na(df, missing_col)

# 결측치 처리가 잘 되었는지 확인해 줍니다.
missing_col = check_missing_col(df) 

## 2.2. Label Encoding

범주(카테고리)형 데이터 존재 시 문자열을 수치형으로 인코딩

In [ ]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i+1  #새로 등장하는 유니크 값들에 대해 1부터 1씩 증가시켜 키값을 부여해줍니다.
            label_maps[col] = label_map
    print(label_maps)
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값(0)으로 채워줍니다.
    return dataframe

df = label_encoder(df, make_label_map(df))

# 3. 모델링

## 3.1. 변수 정의

* X : 독립 변수 (피쳐)
* y : 종속 변수 (타켓)

In [ ]:
X = df.drop(['id', 'Target'], axis=1)
y = df['Target']

## 3.2. 모델 학습


### 3.2.1. 회귀 모델

#### 3.2.1.1. Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression() # 모델 정의
model.fit(X, y) # 학습

#### 3.2.1.2. Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X, y)

### 3.2.2. 분류 모델

#### 3.2.2.1. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X, y)

#### 3.2.2.2. Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X, y)

# 4. 모델 예측

## 4.1 회귀 모델

###  4.1.1. NMAE

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

def nmae(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

y_pred = model.predict(X_test)
print(f'모델 NMAE: {nmae(y_test, y_pred)}')

### 4.1.2. RMSE

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)
RMSE = mean_squared_error(y_test, y_pred)**0.5

## 4.2. 분류 모델

### 4.1.1. Accuracy

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
print('예측 정확도', round(accuracy_score(y_test, y_pred),3))

### 4.1.2. F1-score